# Data Preparation and Cleaning 

In [1]:
#Import libraries
# Standard Packages
import pandas as pd
import numpy as np

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt

# Scipy Stats
import scipy.stats as stats 

# # Statsmodel Api
# import statsmodels.api as sm
# from statsmodels.formula.api import ols

# # SKLearn Modules
# from sklearn.linear_model import LinearRegression
# from sklearn.feature_selection import RFE
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.model_selection import train_test_split
# import sklearn.metrics as metrics

# Suppress future and deprecation warnings
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#Display Full Dataframe width
pd.options.display.max_columns = None
pd.options.display.width = None

### House Sales Dataset

In [2]:
#Import house sales data
kc_house_df = pd.read_csv('/Users/Aidan/Documents/Flatiron/Phase_2/King-County-House-Sales-/data/kc_house_data.csv')

#Let's create a subset that fit the criteria 4 or more bedrooms and no nuisance
kc_family_house_df = kc_house_df[(kc_house_df['bedrooms'] >= 4) & (kc_house_df['nuisance'] == "NO")]

#Reset the index
kc_family_house_df.reset_index(drop=True,inplace= True)

#Preview first 5 rows of subset
kc_family_house_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.190520
1,1180000275,9/29/2021,311000.0,6,2.0,2880,6156,1.0,NO,NO,NO,AVERAGE,Average,7 Average,Gas,PUBLIC,1580,1580,0,0,1956,0,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.225200
2,2944500680,3/17/2022,780000.0,4,2.5,2340,8125,2.0,NO,NO,NO,NONE,Average,8 Good,Gas,PUBLIC,2340,0,440,70,1989,0,"2721 Southwest 343rd Place, Federal Way, Washi...",47.293770,-122.369320
3,2619950340,6/21/2021,975000.0,4,2.5,2980,5859,2.0,NO,NO,NO,AVERAGE,Average,8 Good,Gas,PUBLIC,2980,0,540,170,2011,0,"27950 Northeast 147th Circle, Duvall, Washingt...",47.733170,-121.965305
4,2873000690,6/11/2021,680000.0,4,3.0,2130,7649,1.0,NO,NO,NO,NONE,Very Good,7 Average,Electricity,PUBLIC,1130,1100,440,280,1975,0,"20432 130th Place Southeast, Kent, Washington ...",47.418155,-122.166960


#### Remove Duplicates

In [3]:
#Look for any duplicates in our dataset
kc_family_house_df.duplicated().value_counts()

False    11054
dtype: int64

Looks like there are no duplicates to address.

#### Dealing with Missing Values

In [4]:
#Shape of dataframe (rows, columns)
kc_family_house_df.shape

(11054, 25)

In [5]:
# Dropping any rows with null values, since there is not that many, so it shouldn't really affect the 
# success of our dataset.
kc_family_house_df.dropna(inplace=True)

<ipython-input-5-c290f543c6c0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kc_family_house_df.dropna(inplace=True)


In [6]:
# As well, dropping the 'nuisance' column, since this dataset was filtered to only inlcude rows that
# returned a value of "NO" in our 'nuisance' column.
kc_family_house_df.drop(columns = ['nuisance'], inplace=True)

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
#Shape of dataframe after dropping records with missing values and columns we don't need
kc_family_house_df.shape

(11046, 24)

#### Adding Features

Let's split the address into a new column, zip code.

In [8]:
# Example of an address
kc_family_house_df['address'][0]

'2102 Southeast 21st Court, Renton, Washington 98055, United States'

In [9]:
#Separate the Zip Code from the address
kc_family_house_df["Zip Code"] = kc_family_house_df['address'].apply(lambda x: x.split()[-3][:-1])

<ipython-input-9-aab20c90a77a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kc_family_house_df["Zip Code"] = kc_family_house_df['address'].apply(lambda x: x.split()[-3][:-1])


In [10]:
#Preview our new column
kc_family_house_df["Zip Code"].head()

0    98055
1    98178
2    98023
3    98019
4    98031
Name: Zip Code, dtype: object

#### Remove Outliers

The `column_names.md` file that accompanied the House Sales dataset stated "In some cases due to missing or incorrectly-entered data from the King County Assessor, this API returned locations outside of King County, WA."
<br>
<br>
This means we need to remove houses from our dataset that are not in King County, WA. We pulled zip code information for the county from https://statisticalatlas.com/county/Washington/King-County/Overview.

Compiled King County zip codes into a list to use to subset data frame for only houses that have a zip code in `King_County_WA_zipcodes`.

In [11]:
#View record count before removing Non-King County houses.
kc_family_house_df.shape

(11046, 25)

In [12]:
King_County_WA_zipcodes = ['98001', '98002', '98003', '98004', '98005', '98006', '98007', '98008', '98010', \
    '98011', '98014', '98019', '98021', '98022', '98023', '98024', '98027', '98028', \
    '98029', '98030', '98031', '98032', '98033', '98034', '98038', '98039', '98040', \
    '98042', '98043', '98045', '98047', '98050', '98051', '98052', '98053', '98055', \
    '98056', '98057', '98058', '98059', '98065', '98068', '98070', '98072', '98074', \
    '98075', '98077', '98092', '98101', '98102', '98103', '98104', '98105', '98106', \
    '98107', '98108', '98109', '98112', '98115', '98116', '98117', '98118', '98119', \
    '98121', '98122', '98125', '98126', '98133', '98134', '98136', '98144', '98146', \
    '98148', '98154', '98155', '98158', '98164', '98166', '98168', '98174', '98177', \
    '98178', '98188', '98195', '98198', '98199', '98224', '98288', '98323', '98354', '98391']

In [13]:
kc_family_house_df = kc_family_house_df[kc_family_house_df["Zip Code"].isin(King_County_WA_zipcodes)]

In [14]:
kc_family_house_df.shape

(10941, 25)

We removed 105 house records that were not in King County, WA

In [15]:
kc_family_house_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,Zip Code
0,7399300360,5/24/2022,675000.0,4,1.0,1180,7140,1.0,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1180,0,0,40,1969,0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.190520,98055
1,1180000275,9/29/2021,311000.0,6,2.0,2880,6156,1.0,NO,NO,AVERAGE,Average,7 Average,Gas,PUBLIC,1580,1580,0,0,1956,0,"8504 South 113th Street, Seattle, Washington 9...",47.502045,-122.225200,98178
2,2944500680,3/17/2022,780000.0,4,2.5,2340,8125,2.0,NO,NO,NONE,Average,8 Good,Gas,PUBLIC,2340,0,440,70,1989,0,"2721 Southwest 343rd Place, Federal Way, Washi...",47.293770,-122.369320,98023
3,2619950340,6/21/2021,975000.0,4,2.5,2980,5859,2.0,NO,NO,AVERAGE,Average,8 Good,Gas,PUBLIC,2980,0,540,170,2011,0,"27950 Northeast 147th Circle, Duvall, Washingt...",47.733170,-121.965305,98019
4,2873000690,6/11/2021,680000.0,4,3.0,2130,7649,1.0,NO,NO,NONE,Very Good,7 Average,Electricity,PUBLIC,1130,1100,440,280,1975,0,"20432 130th Place Southeast, Kent, Washington ...",47.418155,-122.166960,98031


Let's export our housing data as a csv to use for prediction modeling.

In [16]:
house_kc_csv = kc_family_house_df.to_csv('house_kc.csv')

### School Districts Dataset

In [17]:
#Import School Districts dataset
schools_df = \
pd.read_csv('/Users/Aidan/Documents/Flatiron/Phase_2/King-County-House-Sales-/data/Report_Card_Assessment_Data_2021-22_School_Year.csv')

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
#Create a new subset of just the King County Schools
king_school_df = schools_df[schools_df.County == "King"].copy()

In [19]:
#Let's look at our school districts
king_school_df.DistrictName.value_counts()

Seattle School District No. 1              27136
Lake Washington School District            14470
Kent School District                       12693
Federal Way School District                12013
Highline School District                    9768
Northshore School District                  9572
Renton School District                      9106
Bellevue School District                    8219
Auburn School District                      8216
Issaquah School District                    7862
Shoreline School District                   5097
Snoqualmie Valley School District           4148
Tahoma School District                      3478
Enumclaw School District                    3264
Tukwila School District                     2342
Riverview School District                   2162
Mercer Island School District               1849
Vashon Island School District               1528
Rainier Prep Charter School District         698
Summit Public School: Atlas                  592
Skykomish School Dis

Technically Fife School District is in both Pierce and King counties. In the `schools_df`, it is listed as being in Pierce County. We are including it so that the King County houses with Zip Codes corresponding to the Fife School District can be included in our overall dataset.

In [20]:
#Create dataframe with King County School Districts (including Fife)
king_school_df = schools_df[(schools_df.County == "King") |\
                            (schools_df.DistrictName == "Fife School District")].copy()

In [21]:
#Let's look at our school districts to see if Fife is included
king_school_df.DistrictName.value_counts()

Seattle School District No. 1              27136
Lake Washington School District            14470
Kent School District                       12693
Federal Way School District                12013
Highline School District                    9768
Northshore School District                  9572
Renton School District                      9106
Bellevue School District                    8219
Auburn School District                      8216
Issaquah School District                    7862
Shoreline School District                   5097
Snoqualmie Valley School District           4148
Tahoma School District                      3478
Enumclaw School District                    3264
Tukwila School District                     2342
Fife School District                        2215
Riverview School District                   2162
Mercer Island School District               1849
Vashon Island School District               1528
Rainier Prep Charter School District         698
Summit Public School

We also want to remove the schools that are not school districts. <br>
<br>
We only care about the overall scores of each school district and not the scores of each demographic. We will subset our dataframe for District Totals for All Students and All Grades. 

In [22]:
king_school_df = king_school_df[(king_school_df.StudentGroup == "All Students") & \
                                (king_school_df.GradeLevel == "All Grades") & \
                                (king_school_df.SchoolName == 'District Total') &\
                               (king_school_df.DistrictName.str.contains("District"))]

In [23]:
#Limited our dataframe for just the columns we care about. 
king_school_df = king_school_df[['DistrictName','TestAdministration','TestSubject','PercentMetStandard']]

In [24]:
#Checking a single school district to see if it has all the scores we want
king_school_df[king_school_df.DistrictName == "Fife School District"]

,DistrictName,TestAdministration,TestSubject,PercentMetStandard
2130,Fife School District,SBAC,Math,30.8%
74381,Fife School District,WCAS,Science,39.0%
219063,Fife School District,AIM,ELA,36.8%
219078,Fife School District,AIM,Math,42.1%
728240,Fife School District,AIM,Science,Suppressed: N<10
728245,Fife School District,SBAC,ELA,46.2%


Looking at this data, we noticed that each district had two scores for each test subject. By researching the Test Administration, we found that the AIM Scores are for kids with cognitive disabilities has far fewer students in their count. We will separate these scores and use the non-AIM scores (SBAC and WCAS) for our overall rating of a school district as it has a higher count of students. 

In [25]:
#SUBSET OF KING COUNTY SCHOOL DISTRICT SCORES FOR KIDS WITH COGNITIVE DISABILITIES (AIM TEST)
cog_dis_king_school_df = king_school_df[king_school_df['TestAdministration'] == 'AIM']

In [26]:
cog_dis_king_school_df.drop(labels='TestAdministration', axis = 1, inplace=True)

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [27]:
cog_dis_king_school_df.head()

,DistrictName,TestSubject,PercentMetStandard
2050,Federal Way School District,Science,14.3%
3521,Mercer Island School District,Math,Suppressed: N<10
5926,Shoreline School District,ELA,44.2%
5942,Shoreline School District,Math,44.2%
7171,Vashon Island School District,ELA,Suppressed: N<10


In [28]:
#LARGER SUBSET OF KING COUNTY SCHOOL DISTRICT SCORES FOR KIDS WITHOUT COGNITIVE DISABILITIES 
main_king_school_df = king_school_df[king_school_df['TestAdministration'] != 'AIM']

In [29]:
main_king_school_df.drop(labels='TestAdministration', axis = 1, inplace=True)

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [30]:
main_king_school_df

,DistrictName,TestSubject,PercentMetStandard
2062,Federal Way School District,ELA,35.6%
2130,Fife School District,Math,30.8%
5384,Renton School District,Math,32.5%
69539,Vashon Island School District,ELA,62.3%
69562,Vashon Island School District,Math,51.3%
...,...,...,...
730736,Enumclaw School District,Science,49.8%
735027,Bellevue School District,Math,65.6%
735906,Auburn School District,ELA,43.6%
735909,Auburn School District,Math,30.2%


#### Pivot Dataset

We want to turn our dataset into a dataframe with `DistrictName` as rows and `TestSubject` as Columns with the corresponding `PercentMetStandard` at their intersection. 

In [31]:
#Create a pivot table
main_king_school_df = main_king_school_df.pivot(index = "DistrictName", columns = "TestSubject",\
                                                values = "PercentMetStandard")

In [32]:
#Reset the index
main_king_school_df= main_king_school_df.reset_index()

In [33]:
#View the columns
main_king_school_df.columns

Index(['DistrictName', 'ELA', 'Math', 'Science'], dtype='object', name='TestSubject')

In [34]:
#View our updated pivot table
main_king_school_df

TestSubject,DistrictName,ELA,Math,Science
0,Auburn School District,43.6%,30.2%,35.1%
1,Bellevue School District,72.3%,65.6%,63.7%
2,Enumclaw School District,56.7%,40.1%,49.8%
3,Federal Way School District,35.6%,21.7%,27.1%
4,Fife School District,46.2%,30.8%,39.0%
5,Highline School District,33.4%,21.5%,26.8%
6,Issaquah School District,76.0%,69.6%,63.2%
7,Kent School District,47.9%,33.7%,39.7%
8,Lake Washington School District,78.5%,70.2%,61.6%
9,Mercer Island School District,82.0%,76.9%,75.9%


It looks like one of the districts is `Rainier Prep Charter School District` which is a charter school and so does not reflect the school districts available based on house location. We will drop this row.

In [35]:
main_king_school_df = main_king_school_df[main_king_school_df['DistrictName']\
                                          != "Rainier Prep Charter School District"]

In [36]:
# Confirming Rainier Prep Charter School District is no longer in our dataframe.
main_king_school_df.index.name = 'index'

In [37]:
main_king_school_df

TestSubject,DistrictName,ELA,Math,Science
index,,,,
0,Auburn School District,43.6%,30.2%,35.1%
1,Bellevue School District,72.3%,65.6%,63.7%
2,Enumclaw School District,56.7%,40.1%,49.8%
3,Federal Way School District,35.6%,21.7%,27.1%
4,Fife School District,46.2%,30.8%,39.0%
5,Highline School District,33.4%,21.5%,26.8%
6,Issaquah School District,76.0%,69.6%,63.2%
7,Kent School District,47.9%,33.7%,39.7%
8,Lake Washington School District,78.5%,70.2%,61.6%


In [38]:
#Resetting index so we have 0 to 19
main_king_school_df.reset_index(drop = True, inplace=True)

In [39]:
#Renaming to remove "TestSubject"
main_king_school_df.rename_axis(None, axis =1, inplace=True)

Create dictionary `King_County_School_District_Zipcodes` with zipcodes for each school district in King County, WA. Information taken from https://statisticalatlas.com/county/Washington/King-County/Overview

In [40]:
King_County_School_District_Zipcodes = {

'Seattle School District No. 1' : ['98101', '98102', '98103', '98104', '98105', '98106', '98107', '98108',\
                                    '98109', '98112', '98115', '98116', '98117', '98118', '98119', '98121',\
                                    '98122', '98125', '98126', '98133', '98134', '98136', '98144', '98146',\
                                    '98154', '98164', '98168', '98174', '98177', '98178', '98195', '98199'],

'Lake Washington School District' : ['98004', '98005', '98007', '98008', '98011', '98033', '98034',\
                                            '98039', '98052', '98053', '98072', '98074', '98075', '98077'],

'Kent School District' : ['98001', '98002', '98010', '98030', '98031', '98032', '98038', '98042', '98055',\
                                 '98058', '98092', '98188', '98198'],

'Federal Way School District' : ['98001', '98003', '98023', '98032', '98198', '98354'],

'Highline School District': ['98032', '98106', '98108', '98126', '98146', '98148', '98158', '98166',\
                                     '98168', '98188', '98198'],

'Northshore School District' : ['98011', '98012', '98021', '98028', '98034', '98036', '98052', '98072',\
                                       '98077', '98155', '98296'],

'Renton School District' : ['98006', '98031', '98032', '98055', '98056', '98057', '98058', '98059',\
                                   '98168', '98178', '98188'],

'Bellevue School District' : ['98004', '98005', '98006', '98007', '98008', '98027', '98033', '98039',\
                                     '98052', '98056', '98059'],

'Auburn School District' : ['98001', '98002', '98010', '98022', '98030', '98032', '98042', '98047',\
                                   '98092', '98391'],

'Issaquah School District': ['98006', '98008', '98024', '98027', '98029', '98038', '98050', '98056',\
                                     '98058', '98059', '98065', '98074', '98075'],

'Shoreline School District' : ['98043', '98133', '98155', '98177'],

'Snoqualmie Valley School District' : ['98014', '98024', '98027', '98038', '98045', '98053', '98065',\
                                              '98068', '98074', '98075', '98224'],

'Tahoma School District' : ['98010', '98027', '98038', '98042', '98045', '98051', '98058',\
                                   '98059', '98065'],

'Enumclaw School District': ['98010', '98022', '98038', '98045', '98051', '98092', '98323', '98391'],

'Tukwila School District' : ['98168', '98178', '98188'],

'Riverview School District' : ['98014', '98019', '98024', '98053', '98065', '98077', '98224'],

'Mercer Island School District' : ['98004', '98006', '98040'],

'Vashon Island School District' : ['98070'],

'Skykomish School District' : ['98045', '98068', '98224', '98288'],

'Fife School District' : ['98001', '98003', '98354', '98371', '98372', '98421', '98422', '98424']
    
}    

In [41]:
#Create For Loop for adding Zip Code column to corresponding School District
for index,district in enumerate(main_king_school_df.DistrictName):
    if district in King_County_School_District_Zipcodes.keys():
        main_king_school_df.loc[index,'zipcodes'] = (",".join(King_County_School_District_Zipcodes[district]))
        

/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/Aidan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a co

In [42]:
#Create new column `zipcodes` which contains all the zipcodes corresponding to each school district
main_king_school_df['zipcodes'] = main_king_school_df['zipcodes'].apply(lambda x: str(x).split(','))

<ipython-input-42-4343d2a3b191>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_king_school_df['zipcodes'] = main_king_school_df['zipcodes'].apply(lambda x: str(x).split(','))


In [43]:
#View our dataframe with our new `zipcodes` column
main_king_school_df

,DistrictName,ELA,Math,Science,zipcodes
0,Auburn School District,43.6%,30.2%,35.1%,"[98001, 98002, 98010, 98022, 98030, 98032, 980..."
1,Bellevue School District,72.3%,65.6%,63.7%,"[98004, 98005, 98006, 98007, 98008, 98027, 980..."
2,Enumclaw School District,56.7%,40.1%,49.8%,"[98010, 98022, 98038, 98045, 98051, 98092, 983..."
3,Federal Way School District,35.6%,21.7%,27.1%,"[98001, 98003, 98023, 98032, 98198, 98354]"
4,Fife School District,46.2%,30.8%,39.0%,"[98001, 98003, 98354, 98371, 98372, 98421, 984..."
5,Highline School District,33.4%,21.5%,26.8%,"[98032, 98106, 98108, 98126, 98146, 98148, 981..."
6,Issaquah School District,76.0%,69.6%,63.2%,"[98006, 98008, 98024, 98027, 98029, 98038, 980..."
7,Kent School District,47.9%,33.7%,39.7%,"[98001, 98002, 98010, 98030, 98031, 98032, 980..."
8,Lake Washington School District,78.5%,70.2%,61.6%,"[98004, 98005, 98007, 98008, 98011, 98033, 980..."
9,Mercer Island School District,82.0%,76.9%,75.9%,"[98004, 98006, 98040]"


### Add New Feature- Comparison Score for School Districts

For comparison purposes, let's create another column with the overall district test score called `District Test Score`

In [44]:
#First we need to strip the "%" sign from the Test Subject scores
main_king_school_df = main_king_school_df.apply(lambda x: x.str.strip('%'))

In [45]:
#Replace Skykomish School District Science value "Suppressed: N<10" with "0" as a placeholder
main_king_school_df.replace(to_replace='Suppressed: N<10', value= 'NaN', inplace=True)

In [46]:
#Turn our ELA, Math, and Science values into floats
main_king_school_df[['ELA','Math','Science']] = main_king_school_df[['ELA','Math','Science']].astype(float)

In [47]:
main_king_school_df

,DistrictName,ELA,Math,Science,zipcodes
0,Auburn School District,43.6,30.2,35.1,NaN
1,Bellevue School District,72.3,65.6,63.7,NaN
2,Enumclaw School District,56.7,40.1,49.8,NaN
3,Federal Way School District,35.6,21.7,27.1,NaN
4,Fife School District,46.2,30.8,39.0,NaN
5,Highline School District,33.4,21.5,26.8,NaN
6,Issaquah School District,76.0,69.6,63.2,NaN
7,Kent School District,47.9,33.7,39.7,NaN
8,Lake Washington School District,78.5,70.2,61.6,NaN
9,Mercer Island School District,82.0,76.9,75.9,NaN


In [48]:
#Create For Loop for adding `District Test Score` column to corresponding Districts
for index, district in enumerate(main_king_school_df['DistrictName']):
    main_king_school_df.loc[index,'District Test Score'] = round((main_king_school_df['ELA'][index] +\
    main_king_school_df['Math'][index] + main_king_school_df['Science'][index])/3, 2)

In [49]:
main_king_school_df

,DistrictName,ELA,Math,Science,zipcodes,District Test Score
0,Auburn School District,43.6,30.2,35.1,NaN,36.30
1,Bellevue School District,72.3,65.6,63.7,NaN,67.20
2,Enumclaw School District,56.7,40.1,49.8,NaN,48.87
3,Federal Way School District,35.6,21.7,27.1,NaN,28.13
4,Fife School District,46.2,30.8,39.0,NaN,38.67
5,Highline School District,33.4,21.5,26.8,NaN,27.23
6,Issaquah School District,76.0,69.6,63.2,NaN,69.60
7,Kent School District,47.9,33.7,39.7,NaN,40.43
8,Lake Washington School District,78.5,70.2,61.6,NaN,70.10
9,Mercer Island School District,82.0,76.9,75.9,NaN,78.27


Need to replace Skykomish School District's total value using the average of the only two scores we have for `ELA` and `Math`

In [50]:
(main_king_school_df['ELA'][15] + main_king_school_df['Math'][15])/2

47.6

In [51]:
main_king_school_df['District Test Score'][15] = \
(main_king_school_df['ELA'][15] + main_king_school_df['Math'][15])/2

<ipython-input-51-a2a8ec8a7121>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_king_school_df['District Test Score'][15] = \


In [52]:
main_king_school_df

,DistrictName,ELA,Math,Science,zipcodes,District Test Score
0,Auburn School District,43.6,30.2,35.1,NaN,36.30
1,Bellevue School District,72.3,65.6,63.7,NaN,67.20
2,Enumclaw School District,56.7,40.1,49.8,NaN,48.87
3,Federal Way School District,35.6,21.7,27.1,NaN,28.13
4,Fife School District,46.2,30.8,39.0,NaN,38.67
5,Highline School District,33.4,21.5,26.8,NaN,27.23
6,Issaquah School District,76.0,69.6,63.2,NaN,69.60
7,Kent School District,47.9,33.7,39.7,NaN,40.43
8,Lake Washington School District,78.5,70.2,61.6,NaN,70.10
9,Mercer Island School District,82.0,76.9,75.9,NaN,78.27


 ## Adding District Name to House Sales Dataset

In [53]:
kc_family_house_df.shape

(10941, 25)

In [54]:
#Create For Loop for adding DistrictName column to corresponding Houses based on Zip Code
for index, zipcode in enumerate(kc_family_house_df['Zip Code']):
    for district in King_County_School_District_Zipcodes.keys():
        if zipcode in King_County_School_District_Zipcodes[district]:
            kc_family_house_df.loc[index, 'DistrictName'] = district

In [56]:
kc_family_house_df.tail()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,Zip Code,DistrictName
10718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fife School District
10738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mercer Island School District
10775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Issaquah School District
10786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bellevue School District
10939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riverview School District


In [57]:
kc_family_house_df.shape

(11053, 26)

In [58]:
kc_family_house_df.isna().sum()

id               112
date             112
price            112
bedrooms         112
bathrooms        112
sqft_living      112
sqft_lot         112
floors           112
waterfront       112
greenbelt        112
view             112
condition        112
grade            112
heat_source      112
sewer_system     112
sqft_above       112
sqft_basement    112
sqft_garage      112
sqft_patio       112
yr_built         112
yr_renovated     112
address          112
lat              112
long             112
Zip Code         112
DistrictName     112
dtype: int64

It seems we added 112 rows onto our dataframe but they seem to have all Null values.

In [59]:
kc_family_house_df.dropna(inplace=True)

In [60]:
kc_family_house_df.shape

(10829, 26)

In [61]:
# 'Auburn School District' : ['98001', '98002', '98010', '98022', '98030', '98032', '98042', '98047',\
#                                '98092', '98391']

In [62]:
kc_family_house_df[kc_family_house_df['DistrictName'] == 'Auburn School District']

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,Zip Code,DistrictName
14,1.329300e+09,11/20/2021,785000.0,4.0,2.5,2640.0,6081.0,2.0,NO,NO,NONE,Average,8 Good,Gas,PUBLIC,2640.0,0.0,420.0,50.0,2012.0,0.0,"12426 Southeast 278th Place, Kent, Washington ...",47.352360,-122.175035,98030,Auburn School District
30,7.697910e+09,4/19/2022,694000.0,4.0,2.0,1660.0,6507.0,2.0,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1660.0,0.0,440.0,100.0,1990.0,0.0,"12031 Southeast 263rd Street, Kent, Washington...",47.365600,-122.179150,98030,Auburn School District
46,7.560100e+09,6/22/2021,740527.0,4.0,5.5,2855.0,6788.0,2.5,NO,NO,NONE,Average,8 Good,Gas,PUBLIC,2855.0,0.0,420.0,160.0,2021.0,0.0,"12230 Southeast 271st Place, Kent, Washington ...",47.358590,-122.177550,98030,Auburn School District
50,5.094400e+09,1/20/2022,755000.0,6.0,5.0,2632.0,12721.0,2.0,NO,NO,NONE,Average,6 Low Average,Gas,PUBLIC,2632.0,0.0,528.0,80.0,2007.0,0.0,"936 24th Street Northeast, Auburn, Washington ...",47.327460,-122.218040,98002,Auburn School District
167,6.852700e+09,1/27/2022,2100000.0,6.0,3.0,3940.0,6000.0,2.0,NO,NO,NONE,Fair,8 Good,Oil,PUBLIC,2840.0,1100.0,0.0,220.0,1900.0,0.0,"505 11th Avenue East, Seattle, Washington 9810...",47.623285,-122.318095,98102,Auburn School District
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10853,1.094000e+09,11/18/2021,730000.0,4.0,2.0,1710.0,10701.0,1.0,NO,NO,NONE,Average,8 Good,Gas,PRIVATE,1710.0,0.0,620.0,0.0,1968.0,0.0,"4615 Northeast 24th Street, Renton, Washington...",47.512730,-122.156860,98059,Auburn School District
10880,1.180005e+09,12/17/2021,570000.0,4.0,2.0,1544.0,6000.0,1.0,NO,NO,NONE,Average,7 Average,Gas,PUBLIC,1544.0,0.0,0.0,260.0,1918.0,1999.0,"8420 South 120th Street, Seattle, Washington 9...",47.495595,-122.226700,98178,Auburn School District
10884,8.026200e+09,9/21/2021,722000.0,4.0,2.0,1660.0,7971.0,1.0,NO,NO,NONE,Very Good,7 Average,Gas,PRIVATE,840.0,820.0,440.0,230.0,1970.0,0.0,"2406 Monterey Avenue Northeast, Renton, Washin...",47.513295,-122.192420,98056,Auburn School District
10915,1.081300e+09,1/26/2022,905000.0,4.0,2.0,1770.0,11250.0,1.0,NO,NO,NONE,Good,8 Good,Gas,PRIVATE,1770.0,0.0,600.0,0.0,1969.0,0.0,"16405 Southeast 145th Street, Renton, Washingt...",47.472655,-122.121375,98059,Auburn School District


 ## Consolidating Dataframes

Joining our school districts dataframe onto the housing data set by `DistrictName` so that every house has a column with it's appropriate school district.

In [63]:
kc_house_school_df = \
kc_family_house_df.merge(main_king_school_df, left_on = 'DistrictName', right_on ='DistrictName')

In [64]:
kc_house_school_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,yr_renovated,address,lat,long,Zip Code,DistrictName,ELA,Math,Science,zipcodes,District Test Score
0,7.399300e+09,5/24/2022,675000.0,4.0,1.0,1180.0,7140.0,1.0,NO,NO,NONE,Good,7 Average,Gas,PUBLIC,1180.0,0.0,0.0,40.0,1969.0,0.0,"2102 Southeast 21st Court, Renton, Washington ...",47.461975,-122.19052,98055,Renton School District,42.9,32.5,33.3,NaN,36.23
1,2.873001e+09,6/11/2021,680000.0,4.0,3.0,2130.0,7649.0,1.0,NO,NO,NONE,Very Good,7 Average,Electricity,PUBLIC,1130.0,1100.0,440.0,280.0,1975.0,0.0,"20432 130th Place Southeast, Kent, Washington ...",47.418155,-122.16696,98031,Renton School District,42.9,32.5,33.3,NaN,36.23
2,5.469701e+09,6/23/2021,810000.0,5.0,3.0,3030.0,24759.0,1.0,NO,NO,NONE,Very Good,8 Good,Gas,PUBLIC,1670.0,1400.0,580.0,90.0,1969.0,0.0,"12605 Southeast 235th Street, Kent, Washington...",47.390790,-122.17303,98031,Renton School District,42.9,32.5,33.3,NaN,36.23
3,7.399301e+09,3/29/2022,728000.0,4.0,2.0,2170.0,7520.0,1.0,NO,NO,NONE,Average,7 Average,Gas,PUBLIC,1240.0,1240.0,490.0,60.0,1973.0,0.0,"1814 Aberdeen Avenue Southeast, Renton, Washin...",47.463930,-122.18974,98055,Renton School District,42.9,32.5,33.3,NaN,36.23
4,8.651500e+09,3/11/2022,2300000.0,4.0,2.5,2370.0,11343.0,2.0,NO,NO,NONE,Average,9 Better,Gas,PUBLIC,2370.0,0.0,600.0,90.0,1984.0,0.0,"3840 206th Place Northeast, Sammamish, Washing...",47.644730,-122.06630,98074,Renton School District,42.9,32.5,33.3,NaN,36.23


Let's export our final dataset as a csv for us to use for modeling. 

In [65]:
final_kc_csv = kc_house_school_df.to_csv('final_kc.csv')